# Dilatometry Offset Calculator Notebook

This notebook is for calculating the offset strain required for steel dilatometry analysis. The calculated offset is equal to the strain if 1% of martensite nucleates in 100% of austenite and is dependent on alloy composition. The fraction of martensite in this calculation can be altered but 1% (i.e., X = 0.01) is generally recommended for continuous cooling transformation (CCT) analysis. 

This method was first developed by Yang and Bhadeshia (H-S. Yang and H.K.D.H. Bhadeshia, Materials Science and Technology 23 (2007) 556-560.). See https://www.tandfonline.com/doi/abs/10.1179/174328407X176857 for more information.

## 1. Input Parameters

The input parameters for this calculation include: **composition**, **compositional_unit**, and **X** - the martensite fraction. 

### Parameter 1: composition

The composition parameter should be inputted in dictionary form and should include values for all alloying elements in the steel. This is because the calculation needs the composition in both mole fraction and mass fraction and thus, when converting from one to the other, the full alloy composition is required. Luckily, the user only needs to input one composition (either mole fraction or mass fraction) but will need to specify which one. Any elements not inputted, that are required in offset calculations, will be assumed to equal 0. The code also includes a fail-safe if the concentration of Fe is not inputted and Fe will be set as 1 - sum(element concentrations).

**Note:** users should ensure the composition is inputted as a mass/mole fraction, and not a percentage.

The alloy composition should be inputted in a dictionary format. The layout of which is:

    dictionary = { key1 : value1, key2: value2, ect. }

An example of an alloy composition in dictionary form is as follows:

    composition = {'Fe':0.9649, 'C':0.004, 'Si':0.003, 'Mn':0.007, 'Ni':0.018, 'Mo':0.003, 'Cr':0.009, 'V':0.0001}

where keys = element symbol, and values = concentration.
   
**Note:** the element symbols are inputted with apostrophes around them (i.e., 'Fe')

### Parameter 2: composition_unit

The second parameter required is simply an input of 2 choices, either:

    composition_unit = 'Mass Fraction'
    composition_unit = 'Mole Fraction'

This parameter is in place to tell the code what units of composition the user has inputted.

### Parameter 3: X

The last parameter is X - which approximately correlates to the percentage transformation being investigated. It is recommended that this parameter is kept as:

    X = 0.01 

for calculating the strain equal to a 1% transformation.

## 2. An Example

An example of how to use the function is presented below:

![title](offset_example.png)

## 3. The Code

In [2]:
def offset_calculator(composition, composition_unit, X):
    
    # A dictionary of the molecular weights of elements #
    molec_weight = {'Fe':55.85,'C':12.01,'Si':28.09,'Mn':54.94,'Ni':58.69,'Cr':52,'Mo':95.94,'W':183.85,'Co':58.93,'V':50.94,'Nb':92.91,'Cu':63.55,'Al':26.98,'Ti':47.88,'O':16,'N':14.01,'B':10.81,'P':30.97,'S':32.06,'Sn':118.71,'As':74.92,'Sb':121.76,'Ca':40.08}

    # Check if the fractions of the required elements have been inputted, and if not set element composition to 0 #
    for element in ['C', 'Mn', 'Si', 'Ni', 'Mo', 'Cr', 'V']:
        if element in composition.keys():
            pass
        else:
            composition[element] = 0

    # Check if Fe has been inputted, and if not add as 1 - sum(elements) #
    if 'Fe' in composition.keys():
        pass
    else:
        composition['Fe'] = 1 - sum(composition.values())
    
    # Create a composition dictionary in mole fraction, if mass fraction inputted
    if composition_unit == 'Mass Fraction':
        mass_frac = composition.copy()
        moles = {}
        for element in mass_frac:
            moles[element] = mass_frac[element]/molec_weight[element]
        total_moles = sum(moles.values())
        mole_frac = {}
        for element in moles:
            mole_frac[element] = moles[element]/total_moles
    
    # Create a composition dictionary in mass fraction, if mole fraction inputted
    elif composition_unit == 'Mole Fraction':
        mole_frac = composition.copy()
        mass = {}
        for element in mole_frac:
            mass[element] = mole_frac[element]*molec_weight[element]
        total_mass = sum(mass.values())
        mass_frac = {}
        for element in mass:
            mass_frac[element] = mass[element]/total_mass
    
    # if 'composition_unit' not selected 
    else:
        return print('Incorrect input for -composition_unit-')

    # Calculate lattice parameter of martensite #
    a_Fe = 0.28664
    a_mart = a_Fe + ((((a_Fe - (0.0279*mole_frac['C']))**2)*(a_Fe + (0.2496*mole_frac['C'])) -a_Fe**3)/(3*(a_Fe**2))) - (0.003*mole_frac['Si']) + (0.006*mole_frac['Mn']) + (0.007*mole_frac['Ni']) + (0.031*mole_frac['Mo']) + (0.005*mole_frac['Cr']) + (0.0096*mole_frac['V'])

    # Calculate lattice parameter of austenite #
    a_aust = 0.3573 + (3.3*10**(-1)*mass_frac['C']) + (9.5*10**(-3)*mass_frac['Mn']) - (2*10**(-3)*mass_frac['Ni']) + (6*10**(-3)*mass_frac['Cr']) + (3.1*10**(-2)*mass_frac['Mo']) + (1.8*10**(-2)*mass_frac['V'])

    # Calculate offset at desired volume fraction, X #
    e = (((a_aust**(-3))*((2*X*(a_mart**3)) + ((1-X)*(a_aust**3))))**(1/3)) - 1
    return e